# Natural Language Processing with classification and vector spaces

# Sentiment Analysis using Logistic Regression

In [2]:
import nltk
from nltk.corpus import twitter_samples

import numpy as np
import pandas as pd

In [3]:
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

Look at the raw texts.

In [6]:
print('The positive tweets : ')
print('\033[92m')
for i in range(5):
  print(all_positive_tweets[i])

The positive tweets : 

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!
@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!
@97sides CONGRATS :)
yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days


In [7]:
print('The negative tweets : ')
print('\033[91m')
for i in range(5):
  print(all_negative_tweets[i])

The negative tweets : 

hopeless for tmr :(
Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(
@Hegelbon That heart sliding into the waste basket. :(
“@ketchBurning: I hate Japanese call him "bani" :( :(”

Me too
Dang starting next week I have "work" :(


## Building helper functions

Process Tweets

In [8]:
import re  # Library for regular opeartions expressions
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer


def process_tweet(tweet):
  
  stemmer = PorterStemmer()
  stopwords_english = stopwords.words('english')

  # remove the stock market tickers like $GE
  tweet = re.sub(r'\$\w*', '', tweet)

  # remove the old style retweet text RT
  tweet = re.sub(r'^RT[\s]+', '', tweet)

  # remove the hyperlinks
  tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

  # removing # symbol
  tweet = re.sub(r'#', '', tweet)

  tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
  tweet_tokens = tokenizer.tokenize(tweet)

  tweets_clean = []

  for word in tweet_tokens:
    if (word not in stopwords_english and
        word not in string.punctuation):
      stem_word = stemmer.stem(word)  #stemming
      tweets_clean.append(stem_word)
  
  return tweets_clean

Build Freqs

In [9]:
def build_freqs(tweets, ys):
  yslist = np.squeeze(ys).tolist()

  freqs = {}
  for y, tweet in zip(yslist, tweets):
    for word in process_tweet(tweet):
      pair = (word, y)
      if pair in freqs:
        freqs[pair]+=1
      else:
        freqs[pair] =1
  return freqs

## Prepare the data

In [10]:
train_pos = all_positive_tweets[:4000]
test_pos = all_positive_tweets[4000:]

train_neg = all_negative_tweets[:4000]
test_neg = all_negative_tweets[4000:]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [11]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [12]:
print('Shape of train_y : ', train_y.shape)
print('Shape of test_y : ', test_y.shape)

Shape of train_y :  (8000, 1)
Shape of test_y :  (2000, 1)


In [13]:
# create a frequency dictionary
freqs = build_freqs(train_x, train_y)

print('Type of freqs : ', type(freqs))
print('Length of freqs : ', len(freqs))

Type of freqs :  <class 'dict'>
Length of freqs :  11346


**Process tweet**

In [14]:
print('This is an example of tweet before processing : \n\n', train_x[0])
print('\nAfter process the tweets :\n\n', process_tweet(train_x[0]))

This is an example of tweet before processing : 

 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

After process the tweets :

 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


## Part 1. Logistic Regression

**Part 1.1 : Sigmoid**

In [15]:
def sigmoid(z):
  h = 1/(1+np.exp(-z))
  return h

**Part 1.2 : Cost Function and Gradient**

In [16]:
def gradientDescent(x, y, theta, alpha, num_iters):
  m = x.shape[0]
  for i in range(num_iters):
    z = np.dot(x, theta)
    h = sigmoid(z)

    # cost function
    J = (-1/m) * (np.dot(y.T, np.log(h)) + np.dot((1-y).T, np.log(1-h)))

    # update the weights theta
    theta = theta - (alpha/m)* (np.dot(x.T, h-y))

  J = float(J)
  return J, theta

## Part 2 : Extracting the features

In [17]:
def extract_features(tweet, freqs):

  # Process the tweet
  word_l = process_tweet(tweet)

  x = np.zeros((1, 3))

  # bias term is set to 1
  x[0, 0] = 1

  for word in word_l:
    x[0, 1] += freqs.get((word, 1), 0)
    x[0, 2] += freqs.get((word, 0), 0)

  assert(x.shape==(1, 3))
  return x

##Part 3 : Training the Model

In [18]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
  X[i, :] = extract_features(train_x[i], freqs)

# training levels corrosponding to X
Y = train_y

# Applying Gradient Descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.24216529.
The resulting vector of weights is [7e-08, 0.0005239, -0.00055517]


## Part 4 : Test the Logistic Regression

In [19]:
def predict_tweet(tweet, freqs, theta):

  x = extract_features(tweet, freqs) 

  # make prediction
  y_pred = sigmoid(np.dot(x, theta))

  return y_pred

In [20]:
def test_logistic_regression(test_x, test_y, feqs, theta):

  # empty list for storing predictions
  y_hat = []

  for tweet in test_x:
    y_pred = predict_tweet(tweet, freqs, theta)

    if y_pred >0.5:
      y_hat.append(1)
    else:
      y_hat.append(0)
    
  accuracy = (y_hat==np.squeeze(test_y)).sum()/len(test_x)
  return accuracy 

In [21]:
accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print('The accuracy of Logistic Regression is :', accuracy)

The accuracy of Logistic Regression is : 0.995


##Part 5 : Predict our own tweet

In [22]:
# Feel free to change the tweet below
my_tweet = 'I am enjoying learning Natural Language Processing #NLP'
print(process_tweet(my_tweet))

y_hat = predict_tweet(my_tweet, freqs, theta)

if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['enjoy', 'learn', 'natur', 'languag', 'process', 'nlp']
Positive sentiment
